__Import__

*   Importieren Sie _fipy_ und benennen Sie das Paket `fp`
*   Fügen Sie zusätzlich
```python
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
```
    hinzu
*   Importieren Sie noch weitere Pakete (z.B. Numpy, ...) nachträglich, falls Sie diese benötigen.

<!--
import fipy as fp
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
-->

*   Importieren Sie die vorbereiteten Klassen `SolveStrategyAbstract` und  `LinearEqSys` des _Strategy_ _Pattern_:
```python
from NSM2_65_AbstractSolver import *
```
*    Zusätzliche Info: Eine kurze Erklärung des Designmusters finden Sie [hier](https://medium.com/@sheikhsajid/design-patterns-in-python-part-1-the-strategy-pattern-54b24897233e)

<!--
from NSM2_65_AbstractSolver import *
-->

__Verfahren des steilsten Abstiegs__

*    Erstellen Sie eine von `SolveStrategyAbstract` abgeleitete Klasse `steepestDescent` und implementieren Sie die abstrakte Methode mit dem Funktionsrumpf
```python
def do(self, A, b, phi):
```
* Programmieren Sie das Verfahren des steilsten Abstiegs

Hinweise:
*    `self` ist das Dummy-Argument und ist das Objekt selbst
*    `A`, `b` und `phi` ist die Matrix, die rechte Seite und die Startlösung jeweils als [`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html)
*    Iterieren Sie z.B. über `500` Iterationen und verwenden Sie die Methode `terminate` von `SolveStrategyAbstract` um auf Konvergenz zu prüfen
*    Verwenden Sie die Methoden [`matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) und [`dot`](https://numpy.org/doc/stable/reference/generated/numpy.dot.html?highlight=dot#numpy.dot) aus `numpy` 

<!--
class steepestDescent(SolveStrategyAbstract):
    def do(self, A, b, phi):
      for i in range(500):
          r = b-np.matmul(A,phi)
          alpha = np.dot(r,r) / (np.dot(r, np.matmul(A,r)))
          phi_o = phi
          phi = phi + alpha * r
          if self.terminate(A,b,phi):
            return phi         
-->

__Testproblem__

*   Definieren Sie das Beispiel aus der Vorlesung
$
\underbrace{
\left[
\begin{matrix} 
1 & 1 \\ 
1 & 2 \\ 
\end{matrix} 
\right]
}_{A}
\underbrace{
\left[
\begin{matrix} 
\phi_1  \\ 
\phi_2 \\ 
\end{matrix} 
\right]
}_{\phi}
=
\underbrace{
\left[ 
\begin{matrix} 
3 \\ 
4 \\ 
\end{matrix} 
\right] 
}_{b}
$
*   Deklarieren Sie die Matrix $A$ und die rechte Seite $b$ als `numpy.ndarray`

<!--
A = np.array([ [1.,1.],[1.,2.] ])
b = np.array([3.,4])
-->

*   Erstellen Sie ein Objekt `SDSolver` von `LinearEqSys` durch den Aufruf des Konstruktors
*   Verwenden Sie als Strategie Ihre Implementierung des steilsten Abstiegs
*   Lösen Sie das System aus der Vorlesung
*   Geben Sie das Residuum `SDSolver.strategy().res_` aus

<!--
SDSolver = LinearEqSys(A, b, steepestDescent())
SDSolver.solve()
print( SDSolver.strategy().res_ )
-->

__Konvektions-Diffusions-Gleichung__

*    Erstellen Sie ein Netz in `fipy` mit `gmsh`. Kopieren Sie dazu den folgenden Befehl:
```python
    geo='''
    Point(1) = {0.0, 0.0, 0.0, 1.0};
    Point(2) = {1.0, 0.0, 0.0, 1.0};
    Point(3) = {1.0, 1.0, 0.0, 1.0};
    Point(4) = {0.0, 1.0, 0.0, 1.0};
    Line(1) = {1, 2};
    Line(2) = {2, 3};
    Line(3) = {3, 4};
    Line(4) = {4, 1};
    Line Loop(1) = {1, 2, 3, 4};
    Plane Surface(1) = {1};
    Physical Line("low") = {3,4};
    Physical Line("up") = {1,2};
    Physical Surface("vol") = {1};
    Transfinite Line {1} = 10 Using Progression 1.0;
    Transfinite Line {2} = 10 Using Progression 1.0;
    Transfinite Line {3} = 10 Using Progression 1.0;
    Transfinite Line {4} = 10 Using Progression 1.0;
    '''
mesh = fp.meshes.Gmsh2D(geo, background=None, communicator=fp.serialComm)
```

*    Sie können sich das Ergebnis durch
```python
    tris = np.zeros((mesh.numberOfCells,3), int)
    for i in range( mesh.numberOfCells ):
        tris[i] = [mesh._cellVertexIDs[0][i], mesh._cellVertexIDs[1][i], mesh._cellVertexIDs[2][i] ]
    plt.close(1)
    plt.figure(1,figsize=(5,5))
    x = np.array(mesh.vertexCoords[0])
    y = np.array(mesh.vertexCoords[1])
    triang = matplotlib.tri.Triangulation(x, y, tris)
    plt.triplot(triang, 'k-')
```
    ansehen

<!--
geo='''
Point(1) = {0.0, 0.0, 0.0, 1.0};
Point(2) = {1.0, 0.0, 0.0, 1.0};
Point(3) = {1.0, 1.0, 0.0, 1.0};
Point(4) = {0.0, 1.0, 0.0, 1.0};
Line(1) = {1, 2};
Line(2) = {2, 3};
Line(3) = {3, 4};
Line(4) = {4, 1};
Line Loop(1) = {1, 2, 3, 4};
Plane Surface(1) = {1};
Physical Line("low") = {3,4};
Physical Line("up") = {1,2};
Physical Surface("vol") = {1};
Transfinite Line {1} = 10 Using Progression 1.0;
Transfinite Line {2} = 10 Using Progression 1.0;
Transfinite Line {3} = 10 Using Progression 1.0;
Transfinite Line {4} = 10 Using Progression 1.0;
'''
mesh = fp.meshes.Gmsh2D(geo, background=None, communicator=fp.serialComm)
tris = np.zeros((mesh.numberOfCells,3), int)
for i in range( mesh.numberOfCells ):
  tris[i] = [mesh._cellVertexIDs[0][i], mesh._cellVertexIDs[1][i], mesh._cellVertexIDs[2][i] ]
plt.close(1)
plt.figure(1,figsize=(5,5))
x = np.array(mesh.vertexCoords[0])
y = np.array(mesh.vertexCoords[1])
triang = matplotlib.tri.Triangulation(x, y, tris)
plt.triplot(triang, 'k-')
-->

*    Definieren Sie die Zellenvariable `phi` in `fipy`
*    Setzen Sie Dirichlet-Randbedingungen jeweils auf die Flächen `low` und `up`
     
     Hinweis: Mit der Methode `mesh.physicalFaces['up']` erhalten Sie eine Liste für die Fläche `up`, die Sie direkt in der Methode `constrain` verwenden können. Analog funktioniert dies auch für `low`.

*    Lösen Sie die Gleichung `eq` $\nabla \cdot \left( \underbrace{1.0}_{\rho} \phi \underbrace{\left[ \begin{matrix} 1.0 \\ 1.0 \\ \end{matrix} \right]}_{U} \right) - \nabla \cdot \left( \underbrace{0.1}_{\Gamma} \nabla \phi \right) = 0$ mit einer zentralen Differenz.
*    Fügen Sie `cacheMatrix()` und `cacheRHSvector()` hinzu, damit Sie die Matrix und die rechte Seite später wieder verwenden können.

<!--
phi = fp.CellVariable(name="phi", mesh=mesh, value=0.)
phi.constrain(1.0, mesh.physicalFaces['low'])
phi.constrain(0.0, mesh.physicalFaces['up'])

Gamma = 0.10
rho = 1.0
U = (1.0,1.0,)
eq = rho * fp.CentralDifferenceConvectionTerm(U) - fp.DiffusionTerm(coeff=Gamma) == 0

eq.cacheMatrix()
eq.cacheRHSvector()
eq.solve( var=phi )
-->

*    Das Ergebnis können Sie durch 
```python
    plt.close(2)
    plt.figure(2,figsize=(6,5))
    plt.tripcolor(triang, np.array(phi.value), vmin=0, vmax=1.0,cmap=plt.cm.jet)
    plt.colorbar()
```
     anschauen.
     
<!--
plt.close(2)
plt.figure(2,figsize=(6,5))
plt.tripcolor(triang, np.array(phi.value), vmin=0, vmax=1.0,cmap=plt.cm.jet)
plt.colorbar()
-->

*   Definieren Sie die Matrix `AA` und die rechte Seite `bb` mit dem System des Konvektions-Diffusions-Problems.

<!--
AA = eq.matrix.numpyArray
bb = eq.RHSvector
-->

*    Erstellen Sie das Objekt `SDSolver` erneut durch einen Konstruktoraufruf.
*    Erstellen Sie sich zwei neue Objekte `CGSolver` und `JacobiSolver` der Klasse `LinearEqSys`.
*    Die jeweiligen Strategien sind `cg` und `jacobi` und sind bereits in `AbstractSolver` implementiert.
*    Verwenden Sie die neue Matrix `AA` und die neue rechte Seite `bb`

<!--
SDSolver = LinearEqSys(AA, bb, steepestDescent())
CGSolver = LinearEqSys(AA, bb, cg())
JacobiSolver = LinearEqSys(AA, bb, jacobi())
-->

*   Lösen Sie das Problem mit den drei unterschiedlichen Lösern.

<!---
SDSolver.solve()
CGSolver.solve()
JacobiSolver.solve()
-->

*  Visualisieren Sie die Residuen durch:

```python
    plt.close(3)
    plt.figure(3)
    plt.plot(SDSolver.strategy().res_, 'k-')
    plt.plot(CGSolver.strategy().res_, 'r-')
    plt.plot(JacobiSolver.strategy().res_, 'g-')
    plt.yscale('log')
```
<!--
    plt.close(3)
    plt.figure(3)
    plt.plot(SDSolver.strategy().res_, 'k-')
    plt.plot(CGSolver.strategy().res_, 'r-')
    plt.plot(JacobiSolver.strategy().res_, 'g-')
    plt.yscale('log')
-->

__Beobachtungen__

*   Was beobachten Sie?
*   Woran liegt das Verhalten?
*   Welche Eigenschaften des Problems müssen Sie ändern?